In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/test.csv')
sample_sub = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
!pip install -U lightautoml

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import torch
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from lightautoml.dataset.roles import CategoryRole

In [ ]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TIMEOUT = 36000
TARGET_NAME = 'target'

In [ ]:
# for reproducibility
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [ ]:
task = Task('binary', metric='auc')

In [ ]:
roles = {
    'drop': ['id'],
    'target': TARGET_NAME
}

In [ ]:
%%time
# Fitting
automl = TabularAutoML(task=task, 
                       timeout=TIMEOUT,
                       cpu_limit=N_THREADS,
                       reader_params={'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                       general_params={'use_algos': [['lgb', 'lgb_tuned', 'cb', 'cb_tuned', 'linear_l2']]}
                      )
automl.fit_predict(train, roles=roles)

In [ ]:
# Prediction
test_pred = automl.predict(test)
sample_sub[TARGET_NAME] = test_pred.data[:, 0]

In [ ]:
fi_score = automl.get_feature_scores('fast').sort_values('Importance', ascending=True)

In [ ]:
plt.figure(figsize=(10, 30))
fi_score.set_index('Feature')['Importance'].plot.barh(fontsize=16)
plt.title('Feature importance', fontsize=18)
plt.show()

In [ ]:
sample_sub.head()

In [ ]:
sample_sub.to_csv('submission.csv', index=False)